### Test1: Mean absolute diff between each group

In [2]:
def timepoint_using_mean_diff(df, group_fields, dose_field, od_field, time_field, time_threshold=3, standardize=False, method='minmax'):
    # Standardize OD values if requested
    if standardize:
        if method == 'zscore':
            df[od_field] = (df[od_field] - df[od_field].mean()) / df[od_field].std()
        elif method == 'minmax':
            df[od_field] = (df[od_field] - df[od_field].min()) / (df[od_field].max() - df[od_field].min())

    # Calculate the mean OD for each group, dose, and time point
    mean_df = df.groupby(group_fields + [dose_field] + [time_field])[od_field].mean().reset_index()
    mean_df = mean_df.rename(columns={od_field: 'mean_raw_od'})

    # Sort the DataFrame so Plate wise absolute diff can be calculated
    mean_df.sort_values(by=(group_fields + [dose_field] + [time_field]), inplace=True)
    # Calculate the absolute difference in mean OD within each plate for successive hours
    mean_df['diff_mean_raw_od'] = mean_df.groupby(group_fields + [dose_field])['mean_raw_od'].diff().abs()

    # Apply the minimum hour threshold
    mean_df = mean_df[mean_df[time_field] >= time_threshold].copy()
    
    # Group by the specified fields and select the row with the highest diff_mean_raw_od
    df_best_timepoints = mean_df.loc[mean_df.groupby(group_fields)['diff_mean_raw_od'].idxmax()]
    # Return relevant columns, including the calculated diff_mean_raw_od
    return df_best_timepoints[group_fields + [time_field]]



import pandas as pd

# timepoint for sf
df = pd.read_csv('data/timepoint_sf.csv')
group_fields = ['Condition','Ratio','Plate']
dose_field = 'XMIC'
od_field = 'Raw_od'
time_field = 'hour'
best_time = timepoint_using_mean_diff(df, group_fields, dose_field ,od_field, time_field, time_threshold=3, standardize=True,method='minmax')


# timepoint for vallo (shud be b.w 9.5 to 10.5)
# df = pd.read_csv('data/timepoint_vallo.csv')
# group_fields = ['Plt']
# dose_field = 'uM'
# od_field = 'RawOD'
# time_field = 'Time_h'
# best_time = timepoint_using_mean_diff(df, group_fields, dose_field ,od_field, time_field, time_threshold=3, standardize=True,method='minmax')


print(best_time)

            Condition  Ratio  Plate      hour
16         20MSynComm     20      1  15.99556
501        20MSynComm     20      2  10.99417
800        20MSynComm     20      3  15.99556
1285       20MSynComm     20      4  10.99389
1584   20MSynComm+ SF  10+10      1  15.99556
2082   20MSynComm+ SF  10+10      2  23.99750
2372   20MSynComm+ SF   2+18      1  19.99639
2758   20MSynComm+ SF   2+18      2  13.99500
3152  20MSynComm+ SFP  10+10      3  15.99556
3542  20MSynComm+ SFP  10+10      4  13.99500
3939  20MSynComm+ SFP   2+18      3  18.99583
4531  20MSynComm+ SFP   2+18      4  22.99722
4807               SF     20      1   4.99250
5248               SF     20      2   4.99250
5494              SFP     20      3   5.99306
5885              SFP     20      4   4.99250


### Test2: Max SNR ratio (not too accurate)

In [ ]:
def timepoint_using_snr(df, condition_fields, dose_field, od_field, time_field, standardize=False, method='zscore', time_threshold=0):
    control_dose = df[dose_field].min()  # assuming the lowest dose as control
    
    # Apply standardization if requested
    if standardize:
        if method == 'zscore':
            df[od_field] = (df[od_field] - df[od_field].mean()) / df[od_field].std()
        elif method == 'minmax':
            df[od_field] = (df[od_field] - df[od_field].min()) / (df[od_field].max() - df[od_field].min())
    
    # Filter out rows based on the minimum time threshold
    df = df[df[time_field] >= time_threshold].copy()
    
    best_timepoints = []

    # Iterate over each unique combination of Condition, Ratio, and Plate
    for condition_values, condition_df in df.groupby(condition_fields):
        snr_by_time = []
        
        # Iterate over each time point within this condition group
        for time, time_df in condition_df.groupby(time_field):
            # Separate control and non-control doses
            control_od = time_df[time_df[dose_field] == control_dose][od_field]
            mean_control_od = control_od.mean()  # baseline signal
            
            # Calculate signal as difference from control for each dose level
            time_df['Signal'] = time_df[od_field] - mean_control_od
            
            # Calculate noise as standard deviation within each dose level
            noise = time_df.groupby(dose_field)[od_field].std().mean()
            
            # Calculate SNR as mean absolute signal over noise
            signal = time_df['Signal'].abs().mean()
            snr = signal / noise if noise != 0 else 0  # avoid division by zero
            
            # Store SNR with the corresponding time point
            snr_by_time.append((time, snr))
        
        # Select the time point with the highest SNR for this condition group
        best_time, best_snr = max(snr_by_time, key=lambda x: x[1])
        
        # Store result with the condition values
        best_timepoints.append((*condition_values, best_time, best_snr))

    # Convert results to a DataFrame for easy viewing
    best_timepoints_df = pd.DataFrame(best_timepoints, columns=condition_fields + ['Best_Timepoint', 'Best_SNR'])
    
    return best_timepoints_df


import pandas as pd

# timepoint for sf
df = pd.read_csv('data/timepoint_sf.csv')
group_fields = ['Condition','Ratio','Plate']
dose_field = 'XMIC'
od_field = 'Raw_od'
time_field = 'hour'
best_timepoints_df = timepoint_using_snr(df, group_fields, dose_field, od_field, time_field, standardize=True,method='minmax', time_threshold=3)


# # timepoint for vallo (shud be b.w 9.5 to 10.5)
# df = pd.read_csv('data/timepoint_vallo.csv')
# group_fields = ['Plt']
# dose_field = 'uM'
# od_field = 'RawOD'
# time_field = 'Time_h'
# best_timepoints_df = timepoint_using_snr(df, group_fields, dose_field, od_field, time_field, standardize=True,method='minmax', time_threshold=3)


print(best_timepoints_df)

          Condition  Ratio  Plate  Best_Timepoint   Best_SNR
0        20MSynComm     20      1        37.00056   2.869877
1        20MSynComm     20      2        38.00083   2.504541
2        20MSynComm     20      3        16.99556   2.550221
3        20MSynComm     20      4        43.00222   3.314093
4    20MSynComm+ SF  10+10      1         5.99278  10.443768
5    20MSynComm+ SF  10+10      2        28.99861   4.330901
6    20MSynComm+ SF   2+18      1         4.99250  11.680565
7    20MSynComm+ SF   2+18      2         4.99250   6.085689
8   20MSynComm+ SFP  10+10      3         6.99306   9.191035
9   20MSynComm+ SFP  10+10      4         6.99306   9.818398
10  20MSynComm+ SFP   2+18      3         5.99306  10.294086
11  20MSynComm+ SFP   2+18      4         5.99306   5.420418
12               SF     20      1         4.99250   5.770851
13               SF     20      2         4.99250   4.903788
14              SFP     20      3         4.99250   6.670171
15              SFP     

### Test 3: Composite scores

In [ ]:
from scipy import stats
import pandas as pd
import numpy as np


def analyze_timepoints_for_drc(df, group_fields, dose_field, od_field, time_field, time_threshold=3, standardize=False, method='minmax'):
    # Create working copy
    data = df.copy()
    
    # Standardize if requested
    if standardize:
        if method == 'zscore':
            data[od_field] = stats.zscore(data[od_field])
        elif method == 'minmax':
            data[od_field] = (data[od_field] - data[od_field].min()) / (data[od_field].max() - data[od_field].min())
    
    # Calculate mean OD for each group, dose, and time
    mean_df = data.groupby(group_fields + [dose_field, time_field])[od_field].agg([
        ('mean_od', 'mean'),
        ('std_od', 'std'),
        ('count', 'count')
    ]).reset_index()
    
    # Calculate metrics for each timepoint
    results = []
    
    for name, group in mean_df.groupby(group_fields + [time_field]):
        # Extract group information
        group_info = dict(zip(group_fields + [time_field], name))
        
        # 1. Signal-to-noise ratio (mean difference between max and min dose / average std)
        max_signal = group['mean_od'].max() - group['mean_od'].min()
        avg_noise = group['std_od'].mean()
        snr = max_signal / avg_noise if avg_noise != 0 else 0
        
        # 2. Dose-response correlation (Spearman correlation between dose and response)
        correlation = stats.spearmanr(group[dose_field], group['mean_od'])[0]
        
        # 3. Coefficient of variation for replicates
        cv = (group['std_od'] / group['mean_od']).mean()
        
        # 4. Dynamic range (ratio of max to min response)
        dynamic_range = group['mean_od'].max() / group['mean_od'].min() if group['mean_od'].min() != 0 else 0
        
        # 5. Mean absolute difference between successive doses
        dose_response_smoothness = np.abs(np.diff(group['mean_od'])).mean()
        
        results.append({
            **group_info,
            'signal_to_noise': snr,
            'dose_response_correlation': correlation,
            'coefficient_of_variation': cv,
            'dynamic_range': dynamic_range,
            'dose_response_smoothness': dose_response_smoothness
        })
    
    results_df = pd.DataFrame(results)
    
    # Filter by time threshold
    results_df = results_df[results_df[time_field] >= time_threshold]
    
    # Calculate composite score (you can adjust weights based on importance)
    weights = {
        'signal_to_noise': 0.3,
        'dose_response_correlation': 0.3,
        'coefficient_of_variation': -0.2,  # Negative because lower is better
        'dynamic_range': 0.1,
        'dose_response_smoothness': 0.1
    }
    
    for metric, weight in weights.items():
        # Normalize each metric to 0-1 scale
        results_df[f'{metric}_normalized'] = (results_df[metric] - results_df[metric].min()) / \
                                           (results_df[metric].max() - results_df[metric].min())
        if weight < 0:
            results_df[f'{metric}_normalized'] = 1 - results_df[f'{metric}_normalized']
    
    # Calculate weighted composite score
    results_df['composite_score'] = sum(
        results_df[f'{metric}_normalized'] * abs(weight)
        for metric, weight in weights.items()
    )
    
    # Select best timepoint for each group based on composite score
    best_timepoints = results_df.loc[results_df.groupby(group_fields)['composite_score'].idxmax()]
    
    return best_timepoints



# timepoint for sf
df = pd.read_csv('data/timepoint_sf.csv')
group_fields = ['Condition','Ratio','Plate']
dose_field = 'XMIC'
od_field = 'Raw_od'
time_field = 'hour'


# Example usage
best_timepoints = analyze_timepoints_for_drc(df, group_fields, dose_field, od_field, time_field, time_threshold=3, standardize=True, method='minmax')
best_timepoints

,Condition,Ratio,Plate,hour,signal_to_noise,dose_response_correlation,coefficient_of_variation,dynamic_range,dose_response_smoothness,signal_to_noise_normalized,dose_response_correlation_normalized,coefficient_of_variation_normalized,dynamic_range_normalized,dose_response_smoothness_normalized,composite_score
39,20MSynComm,20,1,39.00111,10.738079,-0.285714,0.092618,3.248466,0.098010,0.337244,0.394737,0.901422,0.407546,0.778672,0.518500
90,20MSynComm,20,2,41.00167,10.587708,-0.309524,0.107873,3.769806,0.089530,0.332025,0.381579,0.879299,0.512485,0.708920,0.512082
103,20MSynComm,20,3,4.99250,4.259985,0.809524,0.093183,1.468254,0.006238,0.112420,1.000000,0.900603,0.049210,0.023810,0.521149
188,20MSynComm,20,4,41.00167,12.447784,-0.404762,0.104860,4.080871,0.100905,0.396579,0.328947,0.883668,0.575099,0.802482,0.532150
230,20MSynComm+ SF,10+10,1,34.00000,16.532135,-0.380952,0.074389,3.687719,0.105920,0.538327,0.342105,0.927858,0.495962,0.843729,0.583671
278,20MSynComm+ SF,10+10,2,32.99944,13.265312,-0.595238,0.095421,3.963200,0.086513,0.424952,0.223684,0.897357,0.551413,0.684105,0.497614
300,20MSynComm+ SF,2+18,1,5.99278,29.611023,-0.904762,0.032072,3.572165,0.022056,0.992233,0.052632,0.989227,0.472703,0.153924,0.573967
382,20MSynComm+ SF,2+18,2,39.00111,10.096439,-0.428571,0.127311,3.537088,0.099152,0.314975,0.315789,0.851109,0.465642,0.788062,0.484822
399,20MSynComm+ SFP,10+10,3,6.99306,26.274936,-0.690476,0.046398,3.279279,0.025970,0.876453,0.171053,0.968451,0.413748,0.186117,0.567929
448,20MSynComm+ SFP,10+10,4,6.99306,29.834830,-0.595238,0.039284,4.242718,0.015941,1.000000,0.223684,0.978768,0.607677,0.103622,0.633989
